## Imports

In [20]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

## Define the constants

In [21]:
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL="llama3.2"

## Soup up the web scapper


In [22]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url=url
        response = requests.get(url, headers)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title Found"
        for irrlevant in soup.body(["script", "style", "img", "input"]):
            irrlevant.decompose()
        self.text = soup.body.get_text(separator = "\n", strip=True)

In [23]:
site = Website("https://edwarddonner.com")
print(site.title)
print(site.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## time for prompt

In [24]:
system_prompt = """
you are an assistant that analyzes the contents of a website 
and provides a short summary, ignoring text that might be navigation related.
respond in markdown
"""

In [25]:
def prompt_for_user(website):
    user_prompt = f"You are someone very efficient in summarizing texts. You are looking at a website titiled {website.title}"
    user_prompt += """The content of the website is as follows; 
    please provide a concise digestable summary to the this in markdown.
    if it includes any news or announcements, then summarize that too."""
    user_prompt+= f"Now you need to look at the website's content so that you can summarize with \
    the best of your knowledge and generate something very amazing and easy to grasp. The content is as follows: {website.text}"
    return user_prompt

In [26]:
system_prompt

'\nyou are an assistant that analyzes the contents of a website \nand provides a short summary, ignoring text that might be navigation related.\nrespond in markdown\n'

In [27]:
prompt_for_user(site)

"You are someone very efficient in summarizing texts. You are looking at a website titiled Home - Edward DonnerThe content of the website is as follows; \n    please provide a concise digestable summary to the this in markdown.\n    if it includes any news or announcements, then summarize that too.Now you need to look at the website's content so that you can summarize with     the best of your knowledge and generate something very amazing and easy to grasp. The content is as follows: Home\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI

## generate the mesaage to send

In [37]:
def generate_messages(url):
    site = Website(url)
    messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt_for_user(site)}
    ]
    return messages

In [29]:
print(messages)

[{'role': 'system', 'content': '\nyou are an assistant that analyzes the contents of a website \nand provides a short summary, ignoring text that might be navigation related.\nrespond in markdown\n'}, {'role': 'user', 'content': "You are someone very efficient in summarizing texts. You are looking at a website titiled Home - Edward DonnerThe content of the website is as follows; \n    please provide a concise digestable summary to the this in markdown.\n    if it includes any news or announcements, then summarize that too.Now you need to look at the website's content so that you can summarize with     the best of your knowledge and generate something very amazing and easy to grasp. The content is as follows: Home\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly

In [38]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=generate_messages(url)
    )
    return response["message"]["content"]

In [39]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [40]:
display_summary("https://edwarddonner.com")

# Summary
### Edward Donner's Website

**Overview**
Edward Donner is the founder and CTO of Nebula.io, a company applying AI to help people discover their potential. He also co-founded and previously led an AI startup that was acquired in 2021.

**Recent News/Announcements**

* **LLM Expert Courses**: Edward will be launching courses on becoming an LLM expert and leader.
* **2025 AI Executive Briefing**: An upcoming event for executives to learn about the latest AI trends.
* **The Complete Agentic AI Engineering Course**: A course focused on agentic AI engineering, now available.
* **LLM Workshop - Hands-on with Agents**: Resources have been made available for this workshop.

**About**
Edward enjoys writing code, experimenting with LLMs, and DJing. He shares his insights on Hacker News and encourages others to join him in exploring the world of AI.

In [41]:
display_summary("https://www.cnn.com")

This appears to be a sample issue of CNN's website, showcasing a wide range of news articles and topics from various categories such as politics, business, entertainment, sports, and more.

Here are some notable articles and sections highlighted:

1. **Politics**:
	* "Trump's 7 Most Authoritarian Moves So Far"
	* "For Putin Summit, Trump Should Channel Reagan"
2. **Business**:
	* "Alex Jones' Infowars Will Go Up for Sale Yet Again, Judge Rules"
	* "The 'Godfather of AI' Reveals the Only Way Humanity Can Survive Superintelligent AI"
3. **Entertainment**:
	* "Thank You, 'Sex and the City,' for Being Part of Our Story"
	* "Beyoncé Wins Her First Emmy"
4. **Sports**:
	* "Sha'S Carri Richardson Addresses Domestic Violence Arrest and Apologizes to Christian Coleman"
5. **Health & Wellness**:
	* "Guarding is Why You Can't Get Rid of That Ongoing Tension in Your Body"
6. **Celebrity**:
	* "Taylor Swift's Ultimate Hype Man May Be Jason Kelce"
7. **Technology**:
	* "Hear from a Psychiatrist Who Saw a Dozen People Spiral After AI Connections"
8. **Science & Space**:
	* "Meteors Will Stream Across the Sky in One of the Year's Best Celestial Showers"

These are just a few examples of the diverse range of topics and articles covered on this sample issue of CNN's website.